In [73]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import matplotlib.pyplot as plt
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from unidecode import unidecode
import emoji
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementNotInteractableException
from googletrans import Translator
from langdetect import detect

In [2]:
# Set up search parameters
min_price = 1000
max_price = 30000
days_listed = 7
min_mileage = 50000
max_mileage = 200000
min_year = 2000
max_year = 2020
transmission = "automatic"
query = "Honda Civic"
#"102170323157613" = Moncton NB, "114723638540069" = Timmins ON, "111551465530472" = Thunder Bay ON
cities = ["102170323157613", "montreal", "toronto", "114723638540069", "111551465530472", "winnipeg", "regina", "calgary", "vancouver"]

In [3]:
browser = webdriver.Chrome()

# Create a list to store htmls
soups = []

for i, city in enumerate(cities):
    # Set up base url
    base_url = f'https://www.facebook.com/marketplace/{city}/search?'
    #Set up full url
    url = f"{base_url}minPrice={min_price}&maxPrice={max_price}&daysSinceListed={days_listed}&maxMileage={max_mileage}&maxYear={max_year}&minMileage={min_mileage}&minYear={min_year}&transmissionType={transmission}&query={query}&exact=false"

    # Visit the website
    browser.get(url)

    # Find the div element by its CSS selector
    wait = WebDriverWait(browser, 10)
    div_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.x1i10hfl.x1qjc9v5.xjbqb8w.xjqpnuy.xa49m3k.xqeqjp1.x2hbi6w.x13fuv20.xu3j5b3.x1q0q8m5.x26u7qi.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xdl72j9.x2lah0s.xe8uvvx.x11i5rnm.xat24cr.x1mh8g0r.x2lwn1j.xeuugli.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1n2onr6.x16tdsg8.x1hl2dhg.xggy1nq.x1ja2u2z.x1t137rt.x1o1ewxj.x3x9cwd.x1e5q0jg.x13rtm0m.x1q0g3np.x87ps6o.x1lku1pv.x78zum5.x1a2a7pz.x1xmf6yo[tabindex="0"]')))

    # Click on the div element
    div_element.click()

    # Wait for the label element to be clickable
    wait = WebDriverWait(browser, 5)
    
    # Account for variation in label element containing search radius (km)
    try:
        label_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'label[for=":rj:"]')))
    except:
        try:
            label_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'label[for=":re:"]')))
        except:
            try:
                label_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'label[for=":rd:"]')))
            except:
                try:
                    label_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'label[for=":rk:"]')))
                except:
                    label_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'label[for=":rm:"]')))
                

    # Click on the label element
    label_element.click()

    # Press the "Down" key 11 times
    for _ in range(12):
        label_element.send_keys(Keys.DOWN)

    # Press Enter
    label_element.send_keys(Keys.ENTER)

    # Wait for the "Apply" button to be clickable
    wait = WebDriverWait(browser, 10)
    apply_div = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[aria-label="Apply"]')))

    # Click on the div element
    apply_div.click()
    
    # Scroll down to load more results
    scroll_count = 3
    scroll_delay = 1

    for _ in range(scroll_count):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_delay)
    
    # Parse the HTML
    html = browser.page_source
    
    # Create a BeautifulSoup object from the scraped HTML
    soups.append(soup(html, 'html.parser'))
    
    # Check if HTML was scraped correctly
    print(f'finished {city}')
    print('--------------------------------')

finished 102170323157613
--------------------------------
finished montreal
--------------------------------
finished toronto
--------------------------------
finished 114723638540069
--------------------------------
finished 111551465530472
--------------------------------
finished winnipeg
--------------------------------
finished regina
--------------------------------
finished calgary
--------------------------------
finished vancouver
--------------------------------


In [4]:
# Close the browser
browser.quit()

In [5]:
# Create empty lists to store the extracted information
titles_list = []
prices_list = []
mileage_list = []
urls_list = []

# Iterate over the soups
for soup in soups:
    # Extract title information
    titles_div = soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")
    titles_list.extend([title.text.strip() for title in titles_div])
    
    # Extract price information
    prices_div = soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x676frb x1lkfr7t x1lbecb7 x1s688f xzsf02u")
    prices_list.extend([price.text.strip() for price in prices_div])
    
    # Extract mileage information
    mileage_div = soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x3x7a5m x1nxh6w3 x1sibtaa xo1l8bm xi81zsa")
    mileage_list.extend([mileage.text.strip() for mileage in mileage_div])
    
    # Extract URL information
    urls_div = soup.find_all('a', class_="x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1lku1pv")
    urls_list.extend([url.get('href') for url in urls_div])

In [6]:
# Set patterns for mileage and location
mileage_pattern_km = r'(\d+)K km'
mileage_pattern_miles = r'(\d+)K miles'
location_pattern = r'\b[\w\s-]+(?:, \w{2})\b'

# Extract mileage, city and province according to pattern and transform values into integers
mileage_clean = []
city_data = []
province_data = []

# Iterate through mileage list to separate location from mileage
for item in mileage_list:
    match_mileage_km = re.search(mileage_pattern_km, item)
    match_mileage_miles = re.search(mileage_pattern_miles, item)
    match_location = re.search(location_pattern, item)

    # Check for mileage pattern using regex
    if match_mileage_km or match_mileage_miles:
        if match_mileage_km:
            mileage_clean.append(int(match_mileage_km.group(1)) * 1000)
        else:
            mileage_clean.append(int(match_mileage_miles.group(1)) * 1000)
            
    # Check for location pattern using regex     
    elif match_location:
        location = match_location.group(0)
        city, province = map(str.strip, location.split(','))
        city_data.append(city)
        province_data.append(province)
        
    elif item == "":
        mileage_clean.append(0)

# Check for equal length on all lists        
print(len(city_data), len(province_data), len(mileage_clean))
city_data

528 528 528


['Prince',
 'Parrsboro',
 'Moncton',
 'Moncton Parish',
 'Neguac',
 'Lévis',
 'Charlottetown',
 'Québec',
 'Halifax',
 'Halifax',
 'Lunenburg Municipal District',
 'Halifax',
 'East Hants',
 'Halifax',
 'Sherbrooke',
 'Sherbrooke',
 'Ste-Marie',
 'St-Georges',
 'Halifax',
 'Lévis',
 'Sept-Îles',
 'Halifax',
 'Charlottetown',
 'Québec',
 'Shediac Parish',
 'Sherbrooke',
 'Truro',
 'Halifax',
 'Halifax',
 'St-Georges',
 'Québec',
 'Sherbrooke',
 'Halifax',
 'Québec',
 'Halifax',
 'Lévis',
 'Baie-Comeau',
 'Edmundston',
 'Charlottetown',
 'Halifax',
 'Québec',
 'St-Isidore',
 'Kings',
 'Magog',
 'Québec',
 'Thetford Mines',
 'Halifax',
 'Saguenay',
 'Saguenay',
 'Québec',
 'Lévis',
 'St-Georges',
 'Charlottetown',
 'Magog',
 'Fredericton',
 'St-Évariste-de-Forsyth',
 'Fredericton',
 'Québec',
 'Québec',
 'New Richmond',
 'Thetford Mines',
 'Québec',
 'Chandler',
 'Magog',
 'Lévis',
 'Thetford Mines',
 'Saguenay',
 'Québec',
 'Québec',
 'Québec',
 'Halifax',
 'Longueuil',
 'Laval',
 'Montr

In [7]:
# Add all values to a list of dictionaries
vehicles_list = []

for i, item in enumerate(titles_list):
    cars_dict = {}
    
    title_split = titles_list[i].split()
    
    cars_dict["City"] = city_data[i]
    cars_dict["Province"] = province_data[i]
    cars_dict["Year"] = int(title_split[0])
    
    # Use a try-except block to handle model information
    try:
        cars_dict["Make"] = title_split[1]
        cars_dict["Model"] = title_split[2]
    except IndexError:
        # If model information is not present or in a different format, set default values or skip the entry
        cars_dict["Make"] = "Unknown Make"
        cars_dict["Model"] = "Unknown Model"
    
    cars_dict["Price"] = int(re.sub(r'[^\d.]', '', prices_list[i]))
    cars_dict["Mileage"] = mileage_clean[i]
    cars_dict["URL"] = urls_list[i]
    vehicles_list.append(cars_dict)
    
print(vehicles_list)

[{'City': 'Prince', 'Province': 'PE', 'Year': 2015, 'Make': 'Honda', 'Model': 'civic', 'Price': 14500, 'Mileage': 165000, 'URL': '/marketplace/item/818679386477076/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD'}, {'City': 'Parrsboro', 'Province': 'NS', 'Year': 2014, 'Make': 'Honda', 'Model': 'civic', 'Price': 11500, 'Mileage': 155000, 'URL': '/marketplace/item/1331159604450549/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD'}, {'City': 'Moncton', 'Province': 'NB', 'Year': 2016, 'Make': 'Honda', 'Model': 'civic', 'Price': 14444, 'Mileage': 200000, 'URL': '/marketplace/item/1437430427002763/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD'}, {'City': 'Moncton Parish', 'Province': 'NB', 'Year': 2020, 'Make': 'Honda', 'Model': 'civic', 'Price': 26299, 'Mileage': 51000, 'URL': '/marketplace/item/296461122876145/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD'}, {'City': 'Neguac', 'Province': 'NB', 'Year': 2

In [8]:
# Convert to DataFrame
vehicle_df = pd.DataFrame(vehicles_list)
vehicle_df.tail()

,City,Province,Year,Make,Model,Price,Mileage,URL
523,Burnaby,BC,2015,Honda,civic,17985,116000,/marketplace/item/297011986165927/?ref=search&...
524,Surrey,BC,2016,Honda,civic,22000,85000,/marketplace/item/843184004035002/?ref=search&...
525,Surrey,BC,2017,Honda,civic,26500,102000,/marketplace/item/1335130120371836/?ref=search...
526,New Westminster,BC,2019,Honda,civic,17990,94000,/marketplace/item/241226058745517/?ref=search&...
527,Surrey,BC,2012,Honda,civic,12999,152000,/marketplace/item/1262471611081844/?ref=search...


In [9]:
# Drop duplicates based on URL
vehicle_df = vehicle_df.drop_duplicates(subset='URL')
vehicle_df.info()
vehicle_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 416 entries, 0 to 527
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   City      416 non-null    object
 1   Province  416 non-null    object
 2   Year      416 non-null    int64 
 3   Make      416 non-null    object
 4   Model     416 non-null    object
 5   Price     416 non-null    int64 
 6   Mileage   416 non-null    int64 
 7   URL       416 non-null    object
dtypes: int64(3), object(5)
memory usage: 29.2+ KB


,City,Province,Year,Make,Model,Price,Mileage,URL
0,Prince,PE,2015,Honda,civic,14500,165000,/marketplace/item/818679386477076/?ref=search&...
1,Parrsboro,NS,2014,Honda,civic,11500,155000,/marketplace/item/1331159604450549/?ref=search...
2,Moncton,NB,2016,Honda,civic,14444,200000,/marketplace/item/1437430427002763/?ref=search...
3,Moncton Parish,NB,2020,Honda,civic,26299,51000,/marketplace/item/296461122876145/?ref=search&...
4,Neguac,NB,2016,Honda,civic,16000,145000,/marketplace/item/146778901733890/?ref=search&...


In [10]:
# Drop entries with where Mileage = 0
vehicle_df = vehicle_df[vehicle_df['Mileage'] != 0]

# Print information about the cleaned DataFrame
vehicle_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 416 entries, 0 to 527
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   City      416 non-null    object
 1   Province  416 non-null    object
 2   Year      416 non-null    int64 
 3   Make      416 non-null    object
 4   Model     416 non-null    object
 5   Price     416 non-null    int64 
 6   Mileage   416 non-null    int64 
 7   URL       416 non-null    object
dtypes: int64(3), object(5)
memory usage: 29.2+ KB


In [11]:
vehicle_df.head()

,City,Province,Year,Make,Model,Price,Mileage,URL
0,Prince,PE,2015,Honda,civic,14500,165000,/marketplace/item/818679386477076/?ref=search&...
1,Parrsboro,NS,2014,Honda,civic,11500,155000,/marketplace/item/1331159604450549/?ref=search...
2,Moncton,NB,2016,Honda,civic,14444,200000,/marketplace/item/1437430427002763/?ref=search...
3,Moncton Parish,NB,2020,Honda,civic,26299,51000,/marketplace/item/296461122876145/?ref=search&...
4,Neguac,NB,2016,Honda,civic,16000,145000,/marketplace/item/146778901733890/?ref=search&...


In [12]:
#Remove vehicles different from the model on the initial query
model = query.split()[1]

vehicle_df_clean = vehicle_df[vehicle_df['Model'].str.lower() == model.lower()]

vehicle_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 335 entries, 0 to 527
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   City      335 non-null    object
 1   Province  335 non-null    object
 2   Year      335 non-null    int64 
 3   Make      335 non-null    object
 4   Model     335 non-null    object
 5   Price     335 non-null    int64 
 6   Mileage   335 non-null    int64 
 7   URL       335 non-null    object
dtypes: int64(3), object(5)
memory usage: 23.6+ KB


In [13]:
# Add "facebook.com" to the URLs
vehicle_df_clean['URL'] = 'facebook.com' + vehicle_df_clean['URL']
vehicle_df_clean.sample(20)

C:\Users\Victor\AppData\Local\Temp\ipykernel_17888\1054312996.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicle_df_clean['URL'] = 'facebook.com' + vehicle_df_clean['URL']


,City,Province,Year,Make,Model,Price,Mileage,URL
486,Burnaby,BC,2017,Honda,civic,16800,102000,facebook.com/marketplace/item/996290514892913/...
429,Calgary,AB,2015,Honda,civic,16900,149000,facebook.com/marketplace/item/2495156463995714...
497,Coquitlam,BC,2012,Honda,civic,14995,107000,facebook.com/marketplace/item/845608200462388/...
509,Vancouver,BC,2012,Honda,civic,12999,152000,facebook.com/marketplace/item/600482762165456/...
136,Terrebonne,QC,2013,Honda,civic,12495,128000,facebook.com/marketplace/item/1340383700019901...
178,Toronto,ON,2013,Honda,civic,11950,161000,facebook.com/marketplace/item/522251166713513/...
431,Calgary,AB,2013,Honda,civic,17000,124000,facebook.com/marketplace/item/1019507905840435...
413,Calgary,AB,2014,Honda,civic,16999,160000,facebook.com/marketplace/item/1096799738390918...
185,Toronto,ON,2008,Honda,civic,6500,180000,facebook.com/marketplace/item/273577715368760/...
89,Montréal,QC,2014,Honda,civic,9999,169000,facebook.com/marketplace/item/1256629028388483...


In [104]:
# Start the Chrome WebDriver
browser = webdriver.Chrome()

for index, row in vehicle_df_clean.iterrows():
    url = row['URL']
    full_url = f"https://{url}"
    browser.get(full_url)

    # Wait for the "See more" button to be visible and clickable
    try:
        wait = WebDriverWait(browser, 5)
        seemore_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//span[text()="See more"]')))
        seemore_button.click()
        
    except (TimeoutException, NoSuchElementException, ElementNotInteractableException) as e:
        pass

    # Find the <div> element containing the specified text
    target_span_element = wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="xz9dl7a x4uap5 xsag5q8 xkhd6sd x126k92a"]')))

    # Get the text of the target <div> element
    extracted_text = target_span_element.text

    # Remove emojis from the text
    cleaned_text = emoji.demojize(extracted_text)

    # Replace special characters with their regular versions
    cleaned_text = unidecode(cleaned_text)

    # Remove the "[hidden information]" text
    cleaned_text = cleaned_text.replace("[hidden information]", "")

    # Remove the "See less" text
    cleaned_text = cleaned_text.replace("See less", "")
    
    # Remove remaining emogis
    cleaned_text = re.sub(r":(.*?):", "", cleaned_text)

    # Replace newline characters with a single space
    cleaned_text = cleaned_text.replace("\n", " ")

    # Detect the language of the text
    detected_lang = detect(cleaned_text)

    # Create a translator object
    translator = Translator()


    # Translate the text from French to English
    if detected_lang == 'fr':
        # Translate the text from French to English
        cleaned_text = translator.translate(cleaned_text, src='fr', dest='en').text
        
    # Update the "Description" column in the DataFrame
    vehicle_df_clean.at[index, "Description"] = cleaned_text

    print(f'URL:{full_url}')
    print("")
    print(f"Extracted Text{index}:")
    print(cleaned_text)
    print("-----------------------------")


URL:https://facebook.com/marketplace/item/818679386477076/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text0:
Car comes with a new oil change and a new MVI. Has a few minor scratches and dents but is in pretty good condition. Comes with push button start, heated seats, sunroof, Bluetooth, cruise control, alloy wheels, power windows, mirrors and doors, remote start, remote keyless entry, back up camera and the A/C blows ice cold. No accidents, smoke free and pet free. Contact Gordie at  if interested. Will negotiate price in person only. If the ad is up, it's available. 
-----------------------------
URL:https://facebook.com/marketplace/item/1331159604450549/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text1:
Open to reasonable offers, no trades  
-----------------------------
URL:https://facebook.com/marketplace/item/1437430427002763/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text

URL:https://facebook.com/marketplace/item/1475916109821523/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text18:
2010 Honda Civic Sdn DX-G | 1.8L 4Cyl | FWD | Rebuilt Title  197.000KM  1.8L 4Cyl Front Wheel Drive Cruise Control AC AM/ FM/ CD AS IS  Can be seen and test driven at 294 Windmill Road Dartmouth.  We are open: Mon-Fri 9am - 5pm Sat 9am - 2pm  9O2-252-4422 and press 2 (Dartmouth)   #capitalauto #capital auto 
-----------------------------
URL:https://facebook.com/marketplace/item/269392029047088/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text19:
HONDA CIVIC 2019 LX EDITION / INTERIEUR EN TISSU  CHAUFFANT / BLUETOOTH / CAMERA DE RECUL / ROUES 16 POUCES / CRUISE CONTROL ADAPTATIF + ANTI-COLLISION ET CHANGEMENT DE VOIE / AVEC SEULEMENT 64500 KM CERTIFIE ( FEMME UNIQUE PROPRIETAIRE,RAPPORT CARFAX DISPONIBLE) MOTEUR 2.0 LITRES OFFRANT L'EXCELLENTE PUISSANCE DE 158 HP ET 138 LB-FT TOUT EN ETANT DURABLE ET TRES E

URL:https://facebook.com/marketplace/item/827400515729573/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text33:
CIVIC EX CVT   PHARES AUTOMATIQUES TOIT OUVRANT RETROVISEURS CHAUFFANTS, AIR CLIMATISE CONTROLE CLIMATIQUE A DEUX ZONES SIEGES AVANT CHAUFFANTS  SIEGE ELECTRIQUE COTE CONDUCTEUR BLUETOOTH CAMERA DE RECUL COMMANDES AUDIO AU VOLANT DEMARRAGE SANS CLE  KIA Val-Belair      FINANCEMENT DISPONIBLE   NOUS ACHETONS TOUS TYPES DE VEHICULE 
-----------------------------
URL:https://facebook.com/marketplace/item/1307457179913976/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text34:
2017 HONDA CIVIC SPORT HATCHBACK 1.5L   *************$21,900 only 134000KM********** CarFaxFinancingDeliveryWaranty -Keyless entry @ remote start  -Sunroof  -backup and blind spot camera  -Heated seats & steering wheel  -bluetooth apple carplay & aindroid auto   New MVI & Complete Service done   Showroom ready detail  Full tank of gas    
---

URL:https://facebook.com/marketplace/item/1333127080952784/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text45:
HONDA CIVIC SEDAN 2014 ex Low Kilo Roof (NIV 12237A - VID: 9783213) at Thetford Honda.Visit our website for more information.** New arrival washing and inspection in progress !!** For information by text message: text to!Answer the day, evening and weekend!Credit problem?Do you experience a difficult situation or are you lost between the different types of specialized funding?We have the solution for you!Contact us now to find out more!NIV: 2HGFB2F50EH004749;Outdoor color: brown;Interior color: black;Cylinders: 4;Doors: 4;Motor wheels: other;Engine: 1.8L;Transmission: automatic gearbox;This vehicle is sold by Thetford Honda a Thetford Mines inventory published with the help of Marketplace Boost by D2C Media.
-----------------------------
URL:https://facebook.com/marketplace/item/797748485416583/?ref=search&referral_code=null&referral_story_t

URL:https://facebook.com/marketplace/item/945436396527267/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text59:
Honda Civic 2017 LX 4 Portes CVT (NIV H23113A - VID: 9785165) at Honda New Richmond.Visit our website for more information.Easy funding !!Fast delivery, everywhere in Quebec !!!All our used vehicles are full inspectors with a multipoint inspection.Reconditioning used vehicles on the mechanical and aesthetic level.All the information indicated is valid, errors are possible.It is the buyer's responsibility to check: the brand, the model, the options and the guarantee with the manufacturer before the purchase of the vehicle.We guarantee a minimum of 1 key with all of us used vehicles.The owner's manual is not guaranteed with all our NIV vehicles: 2HGFC2F50H027661;Outdoor color: black;Interior color: gray;Cylinders: i4;Doors: 4;Motor wheel: front traction;Engine: 2.0 L;Transmission: automatic gearbox;This vehicle is sold by Honda New Richmond A N

URL:https://facebook.com/marketplace/item/280003224619533/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text77:
Very nice car rolls like nine remote starts winter tires and summer back camera
-----------------------------
URL:https://facebook.com/marketplace/item/659826869370638/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text78:
Very nice car rolls like nine starting winter tires and summer bluetooth
-----------------------------
URL:https://facebook.com/marketplace/item/692170032739078/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text79:
HONDA CIVIC 2012 4 DOORS Clean Automatic Milage 144 013 DISTANCE MODE FASTE ENCONS OR PERFORME -Cruise Control Bluetooth -Distance doors Centralized and Remote -Room -Air Air conditioning well -CD -CD -CD -Electric and heating -Viters Electrics -Porte -PorteElectric tires Summer on original mags possibility of winter tires up to date
-------------

URL:https://facebook.com/marketplace/item/116391651529297/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text99:
Honda Civic 2014, 136343km, 1.8L Excellent condition, low mileage, very reliable and economic, all options: - sunroof.- Recule and browser camera.- Camera on the retro -retrisoners - heated retro -tray - heated seats.- Anti-handling system.- Heated retro -bears.- LED screen.- Speed ​​regulator.- Micro for phone.- Radio: AM, FM, CD, MP3, USB, HDMI, AUX, Bluetooth, steering wheel controls.- TELPURDS ons on the dashboard and driving._ Electric doors and windows before and rear._ Lock centralizes doors and windows._ Air conditions very strong.Numerical adjustment and auto mode.- 4 winter tires A C new._ AA1 engine, AA1 bodywork._ Interior and exterior extremely clean._ No rust point.car rolls like a new one.
-----------------------------
URL:https://facebook.com/marketplace/item/6396770800430128/?ref=search&referral_code=null&referral_story_type=

URL:https://facebook.com/marketplace/item/628546929043631/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text115:
Honda civic ex -L 2009 Automatic only 180,000 kilos Group Electric Group Air Air Air conditioning Bancs in cooking, super clean heated very Economic 1.8 l all Equipped 514 * 609 * 2320 - Mathieu - Hours: Monday to Friday: 9 a.m. to 6 p.m. Saturday: * OnCall * Sunday: * On call * - Address: 5 rue Saint Michel Laval - Website: Automm2016.ca ** Call or text 1 hour before presenting you, Mercii ** 6.495 $
-----------------------------
URL:https://facebook.com/marketplace/item/976692386808994/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text116:
In very good condition 1.7l.Engine A1 Cruise button same thing as turbo
-----------------------------
URL:https://facebook.com/marketplace/item/1473481190146146/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text117:
Bank financing availa

URL:https://facebook.com/marketplace/item/2060424454298106/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD

Extracted Text132:
**** Please don't send me an automatic message "This article is still available" because if you see the announcement it means that it is available **** fr: 2008 Honda Civic184XX KMS Automatic transmission Electric windows AIR REGULATION AIR Air conditioning (but not walking) engine and transmission A1 price is quite firm.Please do not embellish yourself to send me low ball offers, I will not answer in: 2008 honda civic 184xxx kms automatic transmission power windows air conditioning (but not working) cruise control motor and transmission a1 priceIs Pretty Firm.Please don't bother Sending me low ball offers, i won't respond to them.Auto Bano 9800 Boulevard Gouin Ouest Pierrefonds 514xxx607xxx3805
-----------------------------
URL:https://facebook.com/marketplace/item/598510309135104/?ref=search&referral_code=null&referral_story_type=post&__t

TimeoutException: Message: 
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00F0A813+48355]
	(No symbol) [0x00E9C4B1]
	(No symbol) [0x00DA5358]
	(No symbol) [0x00DD09A5]
	(No symbol) [0x00DD0B3B]
	(No symbol) [0x00DFE232]
	(No symbol) [0x00DEA784]
	(No symbol) [0x00DFC922]
	(No symbol) [0x00DEA536]
	(No symbol) [0x00DC82DC]
	(No symbol) [0x00DC93DD]
	GetHandleVerifier [0x0116AABD+2539405]
	GetHandleVerifier [0x011AA78F+2800735]
	GetHandleVerifier [0x011A456C+2775612]
	GetHandleVerifier [0x00F951E0+616112]
	(No symbol) [0x00EA5F8C]
	(No symbol) [0x00EA2328]
	(No symbol) [0x00EA240B]
	(No symbol) [0x00E94FF7]
	BaseThreadInitThunk [0x761800C9+25]
	RtlGetAppContainerNamedObjectPath [0x77387B1E+286]
	RtlGetAppContainerNamedObjectPath [0x77387AEE+238]


In [ ]:
# Close the browser
browser.quit()

In [103]:
vehicle_df_clean.head(120)

,City,Province,Year,Make,Model,Price,Mileage,URL,Description
0,Prince,PE,2015,Honda,civic,14500,165000,facebook.com/marketplace/item/818679386477076/...,Car comes with a new oil change and a new MVI....
1,Parrsboro,NS,2014,Honda,civic,11500,155000,facebook.com/marketplace/item/1331159604450549...,"Open to reasonable offers, no trades"
2,Moncton,NB,2016,Honda,civic,14444,200000,facebook.com/marketplace/item/1437430427002763...,"2016 Honda Civic tinted windows, climate contr..."
3,Moncton Parish,NB,2020,Honda,civic,26299,51000,facebook.com/marketplace/item/296461122876145/...,NEW ARRIVAL! Remote start! One year warranty f...
4,Neguac,NB,2016,Honda,civic,16000,145000,facebook.com/marketplace/item/146778901733890/...,I have this 4door 2016 Honda Civic LX for sale...
...,...,...,...,...,...,...,...,...,...
143,Toronto,ON,2008,Honda,civic,4700,169000,facebook.com/marketplace/item/963268781616993/...,2008 Honda Civic very good condition Kileage 1...
144,Toronto,ON,2013,Honda,civic,9600,163000,facebook.com/marketplace/item/649550630454808/...,NaN
145,Barrie,ON,2012,Honda,civic,2000,185000,facebook.com/marketplace/item/1378803089347427...,NaN
146,Toronto,ON,2010,Honda,civic,6450,173000,facebook.com/marketplace/item/3483477698558755...,NaN


In [ ]:
# Remove special characters and replace with their normal versions
vehicle_df_clean['City'] = vehicle_df_clean['City'].apply(lambda x: unidecode(x))

vehicle_df_clean.head

In [ ]:
# Convert 'Make' and 'Model' columns to title case
vehicle_df_clean['Make'] = vehicle_df_clean['Make'].str.title()
vehicle_df_clean['Model'] = vehicle_df_clean['Model'].str.title()

# Reset the index
vehicle_df_clean = vehicle_df_clean.reset_index(drop=True)

# Display the updated DataFrame
vehicle_df_clean.tail()